# Elasticity equation

In this tutorial we present how to solve the elasticity equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, we want to solve the following problem: find $u$ such that
$$
\nabla \cdot [ 2 \mu \epsilon(u) + \lambda \nabla \cdot u] = -b
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The stress tensor, which can be post-processed from $u$, is given by
$$
    \sigma = 2 \mu \epsilon(u) + \lambda \nabla \cdot u I
$$


## Exercise 6: cantilever beam problem

A cantilever beam problem is when the body is fixed on one of the lateral boundary and subject to a body force.
For this test case we set $\Omega = [0, 2] \times [0, 1]$, the body force equal to $b = [0, -0.01]^\top$ and the following boundary conditions:
$$ 
u = 0 \text{ on } \partial_{left} \Omega 
\qquad 
\nu \cdot \sigma = 0 \text{ on } \partial \Omega \setminus \partial_{left} \Omega 
$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np

import porepy as pp
import pygeon as pg

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [2]:
mesh_size = 0.025
dim = 2

bd_pts = np.array(
    [
        [0, 2, 2, 0],
        [0, 0, 1, 1],
    ]
)
sd = pg.grid_from_boundary_pts(bd_pts, mesh_size, as_mdg=False)
sd.compute_geometry()

With the following code we set the data, in particular the Lamé and the Kirchhoff modulus, and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "elasticity"

lambda_ = 1
mu = 0.5

left = np.hstack([np.isclose(sd.nodes[0, :], 0)] * dim)

fun = lambda _: [0, -1e-2]

data = {pp.PARAMETERS: {key: {"lambda": lambda_, "mu": mu}}}

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
vec_p1 = pg.VecLagrange1(key)

# we construct the matrix
A = vec_p1.assemble_stiff_matrix(sd, data)
# we compute the body force
mass = vec_p1.assemble_mass_matrix(sd)
bd = vec_p1.interpolate(sd, fun)
b = mass @ bd

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [5]:
ls = pg.LinearSystem(A, b)
ls.flag_ess_bc(left, np.zeros(vec_p1.ndof(sd)))
u = ls.solve()

Compute now the stress tensor, one `sd.dim` x `sd.dim` tensor per cell.

In [6]:
cell_sigma = vec_p1.compute_stress(sd, u, data)

# split the tensor in each component for the exporting
cell_sigma_xx = cell_sigma[:, 0, 0]
cell_sigma_xy = cell_sigma[:, 0, 1]
cell_sigma_yy = cell_sigma[:, 1, 1]

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [7]:
# we need to add the z component for the exporting
u = np.hstack((u, np.zeros(sd.num_nodes))).reshape((3, -1))

save = pp.Exporter(sd, "sol", folder_name="ex6")
save.write_vtu(
    [
        ("cell_sigma_xx", cell_sigma_xx),
        ("cell_sigma_xy", cell_sigma_xy),
        ("cell_sigma_yy", cell_sigma_yy),
    ],
    data_pt=[("u", u)],
)

In [10]:
# Consistency check
assert np.isclose(np.linalg.norm(u), 7.068220599848298)
assert np.isclose(np.linalg.norm(cell_sigma), 3.12159609914547)